In [3]:
import tensorflow as tf
from PIL import Image
import numpy as np
from datetime import datetime

In [4]:
#이미지, 상수들
Width=256
Height=144
path="/home/alpha/Dev/과제연구/144p_"
pref1="GrayScale/"
pref2="Colored/"
suff1="_GrayScale.jpg"
suff2="_Colored.jpg"
train_num=500
file_num=867#0
#batch_num=1000

In [106]:
#가중치 초기화 함수
def weight_variable(shape, name):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name=name)
#절편 초기화 함수
def bias_variable(shape, name):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial, name=name)
#2D 컨벌루션 실행
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def getimage(idx):
    img_GS=Image.open(path+pref1+str(idx)+suff1)
    GS_array=np.array(img_GS)[:, :]
    GS_array=np.reshape(GS_array, (Height, Width, 1))
    GS_array=GS_array.astype(np.float32)

    img_CD=Image.open(path+pref2+str(index)+suff2)
    CD_array=np.array(img_CD)[:, :, 0:3]
    CD_array=CD_array.astype(np.float32)
    return GS_array, CD_array

In [6]:
#학습때 사용하는 변수들
x_image = tf.placeholder(np.float32, shape=[None, Height, Width, 1])
y_image = tf.placeholder(np.float32, shape=[None, Height, Width, 3])
#가중치, 절편, 결과
W_conv = weight_variable([10, 10, 1, 3], name='weight')
b_conv = bias_variable([3], name='bias')
y_conv = tf.nn.relu(conv2d(x_image, W_conv)+b_conv)

In [110]:
cost = tf.reduce_sum((y_image-y_conv)*(y_image-y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cost)
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#saver.restore(sess, "01/models.ckpt")

In [45]:
for steps in range(train_num):
    for index in range(file_num):
        img_GS=Image.open(path+pref1+str(index)+suff1)
        GS_array=np.array(img_GS)[:, :]
        GS_array=np.reshape(GS_array, (Height, Width, 1))
        GS_array=GS_array.astype(np.float32)
        
        img_CD=Image.open(path+pref2+str(index)+suff2)
        CD_array=np.array(img_CD)[:, :, 0:3]
        CD_array=CD_array.astype(np.float32)
        sess.run(train_step, feed_dict={x_image:[GS_array], y_image:[CD_array]})
    if(steps%10==0):
        print(sess.run(cost, feed_dict={x_image:[GS_array], y_image:[CD_array]}))
        f=open("01/logs.txt", "a")
        f.write(str(datetime.now())+" | "+str(steps)+" :  "+str(sess.run(cost, feed_dict={x_image:[GS_array], y_image:[CD_array]}))+'\n')
        f.close()
    #if(steps%50==0):
        save_path=saver.save(sess, "01/models.ckpt")

INFO:tensorflow:Restoring parameters from 01/models.ckpt
5.9416e+07


KeyboardInterrupt: 

In [105]:
#pixel_array=CD_array.astype(np.uint8)
#img=Image.frombytes('RGB', (Width, Height), pixel_array);
#img.show()
#print(CD_array[0])
#img_GS.show()
'''
A=np.array(img_GS)
B=np.reshape(GS_array, (Height, Width))
B=B.astype(np.uint8)
Image.fromarray(B, 'L').show()'''

bird=Image.open("person_GS.jpg")
GS_B=np.array(bird)[:, :]
GS_B=np.reshape(GS_B, (Height, Width, 1))
GS_B=GS_B.astype(np.float32)

bird_CD=Image.open("person_CD.jpg")
CD_B=np.array(bird_CD)[:, :, 0:3]
CD_B=CD_B.astype(np.float32)
#Image.fromarray(CD_array, 'RGB').show()
A=sess.run(y_conv, feed_dict={x_image:[GS_B], y_image:[CD_B]})
A=A[0].astype(np.uint8)
B=np.reshape(GS_B, (Height, Width))
B=B.astype(np.uint8)
img = Image.fromarray(A, 'RGB')
img.save('img3.jpg')
img.show()
img1=Image.fromarray(B, 'L')
img1.show()
img1.save('img4.jpg')